# Vision Lab with PYNQ

This notebook shows you how to run the vision example with PYNQ

Let us start by loading a testing image

In [ ]:
import os
home = os.path.expanduser('~') + "/"

from PIL import Image
image = Image.open(home + "xup_compute_acceleration/sources/vision_lab/src/data/fish_wallpaper.jpg")
img_width, img_height = image.size
image.size

## Donwload the accelerator

In [ ]:
import pynq
ol = pynq.Overlay(home + "xup_compute_acceleration/solutions/vision_lab/vision_example.awsxclbin")

Show the available accelerators on the xclbin file

In [ ]:
ol.ip_dict

## Resize kernel

Grab a handler to the resize accelerator and display its signature

In [ ]:
resize = ol.resize_accel_rgb_1
resize.signature

Define the resize value and compute the size of the resulting image

In [ ]:
scale = 5
img_out_height, img_out_width = img_height//scale,  img_width//scale

Compute the size of the input and output images, in bytes, and allocate buffers for them

In [ ]:
import numpy as np
size_in = img_height * img_width * 3
size_out = img_out_height * img_out_width * 3

in_buf = pynq.allocate(size_in, dtype=np.uint8)
out_buf = pynq.allocate(size_out, dtype=np.uint8)

Copy the input image into the accelerator global memory

In [ ]:
in_buf[:] = np.reshape(np.array(image), size_in)
in_buf.sync_to_device()

Run the kernel and wait for its completion

In [ ]:
krnl = resize.start(in_buf, out_buf, img_width, img_height, img_out_width, img_out_height)
krnl.wait()
out_buf.sync_from_device()

Display output image

In [ ]:
result_image = np.reshape(out_buf,(img_out_height, img_out_width, 3))

resize_img = Image.fromarray(result_image, "RGB")

print(f"Output image size {resize_img.size}")
resize_img

## Resize and blur kernel

Grab a handler to the resize and blur kernel

In [ ]:
resize_blur = ol.resize_blur_rgb_1
resize_blur.signature

Define the resize value and compute the size of the resulting image

In [ ]:
scale = 3
img_out_height, img_out_width = img_height//scale,  img_width//scale

size_out = img_out_height * img_out_width * 3

Allocate output image buffer, we can reuse the input image buffer as both kernel access to the same memory bank. You can use the `.args` attribute of the kernel objects to verify this. For instance, `resize_blur.args` 

In [ ]:
out_buf = pynq.allocate(size_out, dtype=np.uint8)

Run the kernel and wait for its completion

In [ ]:
krnl = resize_blur.start(in_buf, out_buf, img_width, img_height, img_out_width, img_out_height, 20.3)
krnl.wait()
out_buf.sync_from_device()

Display output image

In [ ]:
result_image = np.reshape(out_buf,(img_out_height, img_out_width, 3))

resize_blur_img = Image.fromarray(result_image, "RGB")
print(f"Output image size {resize_blur_img.size}")
resize_blur_img

## Release accelerator and memory

In [ ]:
del in_buf
del out_buf
ol.free()

----
Copyright © 2021 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause